# Dataproject - Fertility and Unemployment rate

#### In this data project we are examining whether there are any correlation between unemployment rates and fertility rates on a municipality level in Denmark in the years from 2007 until 2017. First we download the tables of interest from Denmark Statistics (DST) and then merge them into a combined dataset. One this combined dataset we do some graphical explorations of the evolution of the unemployment rate and the fertility rate and the correlation between the two.

In [1]:
# Importing crucial packages

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
import pandas as pd
import pydst
dst = pydst.Dst(lang='en')

## 1 Downloading data from DST
We examine what datasets are available from DST

In [2]:
dst.get_subjects()

,active,desc,hasSubjects,id
0,True,Population and elections,True,02
1,True,Living conditions,True,05
2,True,Education and knowledge,True,03
3,True,Culture and National Church,True,18
4,True,"Labour, income and wealth",True,04
5,True,Prices and consumption,True,06
6,True,National accounts and government finances,True,14
7,True,Money and credit market,True,16
8,True,External economy,True,13
9,True,Business sector in general,True,07


We would like to look at "population and elections" because this is where we can find unformation about fertility rates

In [3]:
dst.get_tables(subjects=['02'])

,active,firstPeriod,id,latestPeriod,text,unit,updated,variables
0,True,2008Q1,FOLK1A,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, marital status, time]"
1,True,2008Q1,FOLK1B,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
2,True,2008Q1,FOLK1C,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, country of origin..."
3,True,2008Q1,FOLK1D,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
4,True,2008Q1,FOLK1E,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, time]"
5,True,1980,FOLK2,2019,Population 1. January,number,2019-02-11 08:00:00,"[age, sex, ancestry, citizenship, country of o..."
6,True,2008,FOLK3,2019,Population 1. January,number,2019-02-11 08:00:00,"[day of birth, birth month, year of birth, time]"
7,True,1769,FT,2019,Population figures from the censuses,number,2019-02-11 08:00:00,"[national part, time]"
8,True,2008,BEF5F,2019,People born in Faroe Islands and living in Den...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"
9,True,2008,BEF5G,2019,People born in Greenland and living in Denmark...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"


We use the table with id 'FOD407', that containts information about fertility rates in each municipality in specific years.

In [4]:
FOD407_vars = dst.get_variables(table_id='FOD407')
FOD407_vars['values'][1][:] ## age= TOT1 to include all ages

[{'id': 'TOT1', 'text': 'Total fertility rate'},
 {'id': '15-19', 'text': '15-19 years'},
 {'id': '20-24', 'text': '20-24 years'},
 {'id': '25-29', 'text': '25-29 years'},
 {'id': '30-34', 'text': '30-34 years'},
 {'id': '35-39', 'text': '35-39 years'},
 {'id': '40-44', 'text': '40-44 years'},
 {'id': '45-49', 'text': '45-49 years'}]

In [5]:
#We are only interested in the total fertility rate, hence we write 'ALDER:[TOT1]'

FOD407 = dst.get_data(table_id = 'FOD407', variables={'OMRÅDE':['*'], 'TID':['*'], 'ALDER':['TOT1'] })
FOD407.sort_values(by=['OMRÅDE', 'TID'], ascending=False)
FOD407.head()

,OMRÅDE,TID,ALDER,INDHOLD
0,All Denmark,2006,Total fertility rate,1847.6
1,Region Hovedstaden,2006,Total fertility rate,1706.2
2,Region Sjælland,2006,Total fertility rate,2061.2
3,Region Syddanmark,2006,Total fertility rate,1928.0
4,Region Midtjylland,2006,Total fertility rate,1919.1


We rename each variable

In [6]:
FOD407_en=FOD407.rename(columns={"OMRÅDE": "AREA", "TID": "YEAR", "INDHOLD": "BIRTH_RATE", "ALDER": "AGE"})
FOD407_en.head(10)

,AREA,YEAR,AGE,BIRTH_RATE
0,All Denmark,2006,Total fertility rate,1847.6
1,Region Hovedstaden,2006,Total fertility rate,1706.2
2,Region Sjælland,2006,Total fertility rate,2061.2
3,Region Syddanmark,2006,Total fertility rate,1928.0
4,Region Midtjylland,2006,Total fertility rate,1919.1
5,Region Nordjylland,2006,Total fertility rate,1902.8
6,Province Byen København,2006,Total fertility rate,1555.0
7,Province Københavns omegn,2006,Total fertility rate,1924.9
8,Province Nordsjælland,2006,Total fertility rate,2143.5
9,Province Bornholm,2006,Total fertility rate,1998.8


In [7]:
#Drops AGE

#FOD407_en.drop(["AGE"], axis=1, inplace=True)
#FOD407_en.head()



We now look at the category "Labour, income and wealth" at Denmark Statistics

In [8]:
dst.get_tables(subjects=['04'])

,active,firstPeriod,id,latestPeriod,text,unit,updated,variables
0,True,2008Q1,FOLK1A,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, marital status, time]"
1,True,2008Q1,FOLK1B,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
2,True,2008Q1,FOLK1C,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, country of origin..."
3,True,2008Q1,FOLK1D,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, citizenship, time]"
4,True,2008Q1,FOLK1E,2019Q1,Population at the first day of the quarter,number,2019-02-11 08:00:00,"[region, sex, age, ancestry, time]"
5,True,1980,FOLK2,2019,Population 1. January,number,2019-02-11 08:00:00,"[age, sex, ancestry, citizenship, country of o..."
6,True,2008,FOLK3,2019,Population 1. January,number,2019-02-11 08:00:00,"[day of birth, birth month, year of birth, time]"
7,True,1769,FT,2019,Population figures from the censuses,number,2019-02-11 08:00:00,"[national part, time]"
8,True,2008,BEF5F,2019,People born in Faroe Islands and living in Den...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"
9,True,2008,BEF5G,2019,People born in Greenland and living in Denmark...,number,2019-02-11 08:00:00,"[sex, age, parents place of birth, time]"


In [9]:
AULP01_vars = dst.get_variables(table_id='AULP01')
AULP01_vars

,elimination,id,map,text,time,values
0,True,OMRÅDE,denmark_municipality_07,region,False,"[{'id': '000', 'text': 'All Denmark'}, {'id': ..."
1,True,ALDER,NaN,age,False,"[{'id': 'TOT', 'text': 'Age, total'}, {'id': '..."
2,True,KØN,NaN,sex,False,"[{'id': 'TOT', 'text': 'Total'}, {'id': 'M', '..."
3,False,Tid,NaN,time,True,"[{'id': '2007', 'text': '2007'}, {'id': '2008'..."


In [10]:
AULP01 = dst.get_data(table_id = 'AULP01', variables={'OMRÅDE':['*'], 'ALDER':['TOT'], 'KØN':['TOT'], 'TID':['*'] })
AULP01.head()

,OMRÅDE,ALDER,KØN,TID,INDHOLD
0,Svendborg,"Age, total",Total,2011,6.8
1,Nordfyns,"Age, total",Total,2011,7.4
2,Langeland,"Age, total",Total,2011,7.9
3,Ærø,"Age, total",Total,2011,4.5
4,Haderslev,"Age, total",Total,2011,6.0


In [11]:
AULP01_en = AULP01.rename(columns={"OMRÅDE": "AREA", "ALDER": "AGE", "KØN": "GENDER",\
                                   "TID": "YEAR", "INDHOLD": "UNEMPLOYMENT_RATE"})
AULP01_en.head(6)

,AREA,AGE,GENDER,YEAR,UNEMPLOYMENT_RATE
0,Svendborg,"Age, total",Total,2011,6.8
1,Nordfyns,"Age, total",Total,2011,7.4
2,Langeland,"Age, total",Total,2011,7.9
3,Ærø,"Age, total",Total,2011,4.5
4,Haderslev,"Age, total",Total,2011,6.0
5,Billund,"Age, total",Total,2011,3.7


In [12]:
AULP01_en.drop(["AGE", "GENDER"], axis=1, inplace=True)
AULP01_en.head()

,AREA,YEAR,UNEMPLOYMENT_RATE
0,Svendborg,2011,6.8
1,Nordfyns,2011,7.4
2,Langeland,2011,7.9
3,Ærø,2011,4.5
4,Haderslev,2011,6.0


We now merge the two different datasets.

In [13]:
merged_data = pd.merge(FOD407_en, AULP01_en, on=['YEAR', 'AREA'], how='left')
merged_data.head(200)

,AREA,YEAR,AGE,BIRTH_RATE,UNEMPLOYMENT_RATE
0,All Denmark,2006,Total fertility rate,1847.6,NaN
1,Region Hovedstaden,2006,Total fertility rate,1706.2,NaN
2,Region Sjælland,2006,Total fertility rate,2061.2,NaN
3,Region Syddanmark,2006,Total fertility rate,1928.0,NaN
4,Region Midtjylland,2006,Total fertility rate,1919.1,NaN
5,Region Nordjylland,2006,Total fertility rate,1902.8,NaN
6,Province Byen København,2006,Total fertility rate,1555.0,NaN
7,Province Københavns omegn,2006,Total fertility rate,1924.9,NaN
8,Province Nordsjælland,2006,Total fertility rate,2143.5,NaN
9,Province Bornholm,2006,Total fertility rate,1998.8,NaN


We now drop the year 2006 and 2018 since, we don't have any information about the unemployment rate in these years. Also the fertility rates for the municipalities Læsø, Samsø, Ærø, Fanø and Christiansø are not given because of to few observations, so these are dropped from the data set as well. 

In [115]:
merged_data = merged_data.dropna()
merged_data = merged_data[merged_data['BIRTH_RATE'] != '..']
### HUSK AT DROPPE REGIONER ETC.!

for val in ['Region', 'Province', 'All Denmark']: 
    I = merged_data.AREA.str.contains(val)
    merged_data = merged_data.loc[I == False] # keep everything else

#Checking if they have been dropped
print(merged_data.AREA.unique())
print(merged_data.AREA.nunique())

['Copenhagen' 'Frederiksberg' 'Dragør' 'Tårnby' 'Ballerup' 'Brøndby'
 'Gentofte' 'Gladsaxe' 'Glostrup' 'Herlev' 'Albertslund' 'Hvidovre'
 'Høje-Taastrup' 'Lyngby-Taarbæk' 'Rødovre' 'Ishøj' 'Vallensbæk' 'Furesø'
 'Allerød' 'Fredensborg' 'Helsingør' 'Hillerød' 'Hørsholm' 'Rudersdal'
 'Egedal' 'Frederikssund' 'Halsnæs' 'Gribskov' 'Bornholm' 'Greve' 'Køge'
 'Roskilde' 'Solrød' 'Lejre' 'Odsherred' 'Holbæk' 'Faxe' 'Kalundborg'
 'Ringsted' 'Slagelse' 'Stevns' 'Sorø' 'Lolland' 'Næstved' 'Guldborgsund'
 'Vordingborg' 'Middelfart' 'Assens' 'Faaborg-Midtfyn' 'Kerteminde'
 'Nyborg' 'Odense' 'Svendborg' 'Nordfyns' 'Langeland' 'Haderslev'
 'Billund' 'Sønderborg' 'Tønder' 'Esbjerg' 'Varde' 'Vejen' 'Aabenraa'
 'Fredericia' 'Kolding' 'Vejle' 'Horsens' 'Syddjurs' 'Norddjurs'
 'Favrskov' 'Odder' 'Randers' 'Silkeborg' 'Skanderborg' 'Aarhus'
 'Hedensted' 'Herning' 'Holstebro' 'Lemvig' 'Struer' 'Ikast-Brande'
 'Ringkøbing-Skjern' 'Skive' 'Viborg' 'Morsø' 'Thisted' 'Brønderslev'
 'Frederikshavn' 'Vesthimmerl

Birth_rate to numeric

In [116]:
merged_data['BIRTH_RATE_F'] = pd.to_numeric(merged_data['BIRTH_RATE'])
merged_data['BIRTH_RATE_F']

133     1547.0
134     1692.6
135     2109.5
136     2227.0
137     1847.4
138     2015.5
139     1956.2
140     2023.0
141     1841.5
142     1607.2
143     1696.2
144     1841.2
145     2104.1
146     1803.6
147     2096.0
148     1896.6
149     2139.7
150     2148.8
151     2486.3
152     2194.9
153     1976.7
154     1970.9
155     1915.8
156     2124.8
157     2246.1
158     2170.5
159     2148.8
160     2273.7
161     2105.2
163     1971.3
         ...  
1362    2018.1
1363    1992.3
1364    2325.2
1365    1993.1
1366    1797.3
1367    1844.4
1368    1818.2
1369    2055.3
1370    2211.6
1371    2033.1
1372    1929.5
1373    1971.8
1374    1828.4
1375    1751.4
1376    2185.9
1377    1842.5
1378    1650.9
1379    1689.4
1380    1913.2
1381    1998.8
1382    1989.0
1383    1746.3
1384    1677.6
1385    2053.0
1386    2126.1
1387    1923.3
1388    1693.2
1389    1833.5
1390    1730.4
1391    1837.7
Name: BIRTH_RATE_F, Length: 1034, dtype: float64

# Analysis

In [117]:
## INDSÆT KORT HER

In [118]:
# Laver ny variable for birth_rate index
merged_data1 = merged_data.copy()
merged_data1 = merged_data1.set_index("YEAR")

merged_data1['FERTILITY INDEX'] = merged_data1.groupby('AREA')['BIRTH_RATE_F'].transform(lambda x: x/x[2007])
merged_data1.head()


,AREA,AGE,BIRTH_RATE,UNEMPLOYMENT_RATE,BIRTH_RATE_F,FERTILITY INDEX
YEAR,,,,,,
2007,Copenhagen,Total fertility rate,1547.0,5.8,1547.0,1.0
2007,Frederiksberg,Total fertility rate,1692.6,4.0,1692.6,1.0
2007,Dragør,Total fertility rate,2109.5,2.3,2109.5,1.0
2007,Tårnby,Total fertility rate,2227.0,3.5,2227.0,1.0
2007,Ballerup,Total fertility rate,1847.4,2.5,1847.4,1.0


In [119]:
# Laver ny variable for unemployment index
merged_data1['UNEMPLOYMENT INDEX'] = merged_data1.groupby('AREA')['UNEMPLOYMENT_RATE'].transform(lambda x: x/x[2007])
merged_data1.head()


,AREA,AGE,BIRTH_RATE,UNEMPLOYMENT_RATE,BIRTH_RATE_F,FERTILITY INDEX,UNEMPLOYMENT INDEX
YEAR,,,,,,,
2007,Copenhagen,Total fertility rate,1547.0,5.8,1547.0,1.0,1.0
2007,Frederiksberg,Total fertility rate,1692.6,4.0,1692.6,1.0,1.0
2007,Dragør,Total fertility rate,2109.5,2.3,2109.5,1.0,1.0
2007,Tårnby,Total fertility rate,2227.0,3.5,2227.0,1.0,1.0
2007,Ballerup,Total fertility rate,1847.4,2.5,1847.4,1.0,1.0


In [128]:
#UNEMPLOYMENT RATE

def interactive_figure(fokus):
    # Choose nationality for the pie chart 
    merged_data2 = merged_data1[merged_data1.AREA.isin([fokus])]


    # Sum the observations grouped by the regions and plot the chart
    merged_data2.groupby('AREA')['UNEMPLOYMENT INDEX', 'FERTILITY INDEX'].plot(legend=True);
    plt.title('Development in fertilityrate and unemployment rate 2007-2017')
    #plt.figure(figsize=(20,10))
    plt.show()

liste=merged_data1.AREA.unique()

widgets.interact(interactive_figure,
    fokus=widgets.Dropdown(description="$Municipal$", options=liste, value='Slagelse'),);

interactive(children=(Dropdown(description='$Municipal$', index=39, options=('Copenhagen', 'Frederiksberg', 'D…

In [ ]:
#CHANGE COLORS OF THE PLOTS

In [ ]:
#DETTE KODE SKAL SLETTES!
#FOD407_en_group = FOD407_en.groupby(['AREA','YEAR'])['BIRTH_RATE'].sum().reset_index()
#FOD407_en_group.head()